In [0]:
simpleData = [("James","Sales","NY",90000,34,10000), \
    ("Michael","Sales","NY",86000,56,20000), \
    ("Robert","Sales","CA",81000,30,23000), \
    ("Maria","Finance","CA",90000,24,23000), \
    ("Raman","Finance","CA",99000,40,24000), \
    ("Scott","Finance","NY",83000,36,19000), \
    ("Jen","Finance","NY",79000,53,15000), \
    ("Jeff","Marketing","CA",80000,25,18000), \
    ("Kumar","Marketing","NY",91000,50,21000) \
  ]
columns= ["employee_name","department","state","salary","age","bonus"]
 
df = spark.createDataFrame(data = simpleData, schema = columns)
df.printSchema()
df.show(truncate=False)

In [0]:
from pyspark.sql.functions import col

df.sort("department").show(truncate=False)
df.sort("department","state").show(truncate=False)


In [0]:
df.sort(df.department.asc(),df.state.asc()).show(truncate=False)

In [0]:
df.orderBy("department","state").show(truncate=False)
df.orderBy(col("department").asc(),col("state").asc()).show(truncate=False)

In [0]:
simpleData = [("James","Sales","NY",90000,34,10000), \
    ("Michael","Sales","NY",86000,56,20000), \
    ("Robert","Sales","CA",81000,30,23000), \
    ("Maria","Finance","CA",90000,24,23000) \
  ]
 
columns= ["employee_name","department","state","salary","age","bonus"]
df = spark.createDataFrame(data = simpleData, schema = columns)
 
simpleData2 = [("James","Sales","NY",90000,34,10000), \
    ("Maria","Finance","CA",90000,24,23000), \
    ("Jen","Finance","NY",79000,53,15000), \
    ("Jeff","Marketing","CA",80000,25,18000), \
    ("Kumar","Marketing","NY",91000,50,21000) \
  ]

columns2= ["employee_name","department","state","salary","age","bonus"]
df2 = spark.createDataFrame(data = simpleData2, schema = columns2)

In [0]:
full_df = df.union(df2)
full_df.display()

In [0]:
full_df2 = df.union(df2).dropDuplicates(["employee_name"])
full_df2.display()

In [0]:
df1 = spark.createDataFrame(data = [[1,2,3]], schema = ["col1", "col2", "col3"])
df2 = spark.createDataFrame(data = [[2,3,4]], schema = ["col2", "col3", "col4"])

df3 = df1.unionByName(df2, allowMissingColumns=True)
df3.display()

In [0]:
emp = [(1,"Smith",-1,"2018","10","M",3000), \
    (2,"Rose",1,"2010","20","M",4000), \
    (3,"Williams",1,"2010","10","M",1000), \
    (4,"Jones",2,"2005","10","F",2000), \
    (5,"Brown",2,"2010","40","",-1), \
      (6,"Brown",2,"2010","50","",-1) \
  ]
empColumns = ["emp_id","name","superior_emp_id","year_joined", \
       "emp_dept_id","gender","salary"]

emp_df = spark.createDataFrame(data=emp, schema = empColumns)

dept = [("Finance",10), \
    ("Marketing",20), \
    ("Sales",30), \
    ("IT",40) \
  ]
deptColumns = ["dept_name","dept_id"]

dept_df = spark.createDataFrame(data=dept, schema = deptColumns)

Supported join types include: 'inner', 'outer', 'full', 'fullouter', 'full_outer', 'leftouter', 'left', 'left_outer', 'rightouter', 'right', 'right_outer', 'leftsemi', 'left_semi', 'semi', 'leftanti', 'left_anti', 'anti', 'cross'.

In [0]:
inner_join_df = emp_df.join(dept_df, emp_df.emp_dept_id == dept_df.dept_id, "inner")
inner_join_df.display()

In [0]:
left_join_df = emp_df.join(dept_df, emp_df.emp_dept_id == dept_df.dept_id, "left")
left_join_df.display()

In [0]:
left_semi_join_df = emp_df.join(dept_df, emp_df.emp_dept_id == dept_df.dept_id, "left_semi")
left_semi_join_df.display()

In [0]:
left_anti_join_df = emp_df.join(dept_df, emp_df.emp_dept_id == dept_df.dept_id, "left_anti")
left_anti_join_df.display()

In [0]:
right_join_df = emp_df.join(dept_df, emp_df.emp_dept_id == dept_df.dept_id, "right")
right_join_df.display()

In [0]:
outer_join_df = emp_df.join(dept_df, emp_df.emp_dept_id == dept_df.dept_id, "outer")
outer_join_df.show(truncate=False)

In [0]:
for dept in dept_df.collect():
  print(dept["dept_name"], dept["dept_id"])


In [0]:
dept_df.cache()
dept_df.is_cached

In [0]:
dept_df.unpersist()
dept_df.is_cached

In [0]:
dept_df.display()

In [0]:
# a function to split the string on space and convert the first alpahbet in the words into uppercase
def split_and_uppercase(s):
  return ' '.join([w[0].upper() + w[1:] for w in s.split(' ')])

from pyspark.sql.types import StringType

udf_split_and_uppercase = udf(split_and_uppercase, StringType())

columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders")]
 
df = spark.createDataFrame(data=data, schema=columns)
df.show()
df.select(df.Seqno, udf_split_and_uppercase(df.Name).alias("Name upper")).show()
df.show()
# split_and_uppercase("test sun")

In [0]:
@udf("string")
def uppercase(s):
  return s.upper()

spark.udf.register("uppercase", uppercase)

df.withColumn("Name upper", uppercase(df.Name)).show()

In [0]:
df.createOrReplaceTempView("dfTable")

In [0]:
spark.sql("select * from dfTable").show()

In [0]:
df.createOrReplaceGlobalTempView("dfTableG")

In [0]:
spark.sql("select * from global_temp.dfTableG").show()

In [0]:
spark.sql("SHOW VIEWS IN global_temp;").show()

In [0]:
# use the uppercase function the sql query
spark.sql("select Seqno, uppercase(Name) as Name_upper from global_temp.dfTableG").show()

In [0]:
data = [("James","M",60000),("Michael","M",70000),
        ("Robert",None,400000),("Maria","F",500000),
        ("Jen","",None)]
 
columns = ["name","gender","salary"]

df = spark.createDataFrame(data = data, schema = columns)
df.show()

In [0]:
from pyspark.sql.functions import when

df.withColumn("new_gender", 
              when(df.gender.isNull(), "")
              .when(df.gender == "M", "Male")
              .when(df.gender == "F", "Female")
              .otherwise("U")).show()

In [0]:

# to fill in the null values with a default value
df.na.fill({"gender": "U"}).show()

In [0]:
from pyspark.sql.functions import col, isnull

df.filter(isnull(col("gender"))).show()

In [0]:
from pyspark.sql.functions import when
df = df.withColumn("gender", 
              when(df.gender == "", "U")
              .otherwise(df.gender))

df.show()


In [0]:
df = df.na.replace(["M", "F"], ["Male", "Female"], subset=["gender"])
df = df.na.fill("U", subset=["gender"])
display(df)